In [ ]:
'/Users/Leon/Library/CloudStorage/GoogleDrive-leoactuario@ciencias.unam.mx/Mi unidad/maestria/ia/segundo semestre/Visión compuacional/Practica Clasificador Bayesiano/Comida.rar'

In [3]:
import rarfile
import os

def extraer_rar(archivo_rar, destino='.'):
    if not rarfile.is_rarfile(archivo_rar):
        print(f"{archivo_rar} no es un archivo .rar válido.")
        return

    try:
        with rarfile.RarFile(archivo_rar) as rf:
            print(f"Extrayendo archivos de: {archivo_rar}")
            rf.extractall(path=destino)
            print(f"Archivos extraídos en: {destino}")
    except rarfile.Error as e:
        print(f"Ocurrió un error: {e}")

# Ejemplo de uso
ruta_rar = '/Users/Leon/Library/CloudStorage/GoogleDrive-leoactuario@ciencias.unam.mx/Mi unidad/maestria/ia/segundo semestre/Visión compuacional/Practica Clasificador Bayesiano/Comida.rar'
directorio_destino = '/Users/Leon/Library/CloudStorage/GoogleDrive-leoactuario@ciencias.unam.mx/Mi unidad/maestria/ia/segundo semestre/Visión compuacional/Practica Clasificador Bayesiano/extraido'
os.makedirs(directorio_destino, exist_ok=True)

extraer_rar(ruta_rar, directorio_destino)


Extrayendo archivos de: /Users/Leon/Library/CloudStorage/GoogleDrive-leoactuario@ciencias.unam.mx/Mi unidad/maestria/ia/segundo semestre/Visión compuacional/Practica Clasificador Bayesiano/Comida.rar
Ocurrió un error: Failed the read enough data: req=65536 got=51


In [2]:
import os

ruta_actual = os.getcwd()
print("Estás en la ruta:", ruta_actual)


Estás en la ruta: /Users/Leon/Documents/GitHub/Vision-computacional/Practica Clasificador Bayesiano


In [ ]:
'/Users/Leon/Documents/GitHub/Vision-computacional/Practica' 
'/Users/Leon/Library/CloudStorage/GoogleDrive-leoactuario@ciencias.unam.mx/Mi\ unidad/maestria/ia/segundo semestre/Visión compuacional/Practica Clasificador Bayesiano/Comida.rar'
'/Users/Leon/Library/CloudStorage/GoogleDrive-leoactuario@ciencias.unam.mx/Mi unidad/maestria/ia/segundo semestre/Visión compuacional/Practica Clasificador Bayesiano/Comida.rar'
/Users/Leon/Library/CloudStorage/GoogleDrive-leoactuario@ciencias.unam.mx/Mi unidad/maestria/ia/segundo semestre/Visión compuacional/Practica Clasificador Bayesiano

In [4]:
pip install labelme

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.4 MB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 7.0 MB 34.5 MB/s eta 0:00:01
     |████████████████████████████████| 7.7 MB 13.8 MB/s eta 0:00:01
     |████████████████████████████████| 184 kB 17.0 MB/s eta 0:00:01
     |████████████████████████████████| 14.1 MB 9.7 MB/s eta 0:00:011
     |████████████████████████████████| 61 kB 343 kB/s  eta 0:00:01
     |████████████████████████████████| 2.2 MB 13.3 MB/s eta 0:00:01
     |████████████████████████████████| 431 kB 15.4 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 8.3 MB/s  eta 0:00:01
     |████████████████████████████████| 16.8 MB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 6.2 MB 13.2 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 7.7 MB/s  eta 0:00:01
     |████████████████████████████████| 417 kB 14.2 MB/s eta 0:00:01
     |█████████████████████